In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 6: Universal Orbit Map ---</h2><p>You've landed at the Universal Orbit Map facility on Mercury.  Because navigation in space often involves transferring between orbits, the orbit maps here are useful for finding efficient routes between, for example, you and Santa. You download a map of the local orbits (your puzzle input).</p>
<p>Except for the universal Center of Mass (<code>COM</code>), every object in space is in orbit around <span title="What do you mean, Kerbal Space Program doesn't have accurate orbital physics?">exactly one other object</span>.  An <a href="https://en.wikipedia.org/wiki/Orbit">orbit</a> looks roughly like this:</p>
<pre><code>                  \
                   \
                    |
                    |
AAA--&gt; o            o &lt;--BBB
                    |
                    |
                   /
                  /
</code></pre>
<p>In this diagram, the object <code>BBB</code> is in orbit around <code>AAA</code>. The path that <code>BBB</code> takes around <code>AAA</code> (drawn with lines) is only partly shown. In the map data, this orbital relationship is written <code>AAA)BBB</code>, which means "<code>BBB</code> is in orbit around <code>AAA</code>".</p>
<p>Before you use your map data to plot a course, you need to make sure it wasn't corrupted during the download.  To verify maps, the Universal Orbit Map facility uses <em>orbit count checksums</em> - the total number of <em>direct orbits</em> (like the one shown above) and <em>indirect orbits</em>.</p>
	<p>Whenever <code>A</code> orbits <code>B</code> and <code>B</code> orbits <code>C</code>, then <code>A</code> <em>indirectly orbits</em> <code>C</code>.  This chain can be any number of objects long: if <code>A</code> orbits <code>B</code>, <code>B</code> orbits <code>C</code>, and <code>C</code> orbits <code>D</code>, then <code>A</code> indirectly orbits <code>D</code>.
</p><p>For example, suppose you have the following map:</p>
<pre><code>COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
</code></pre>
<p>Visually, the above map of orbits looks like this:</p>
<pre><code>        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I
</code></pre>
<p>In this visual representation, when two objects are connected by a line, the one on the right directly orbits the one on the left.</p>
<p>Here, we can count the total number of orbits as follows:</p>
<ul>
<li><code>D</code> directly orbits <code>C</code> and indirectly orbits <code>B</code> and <code>COM</code>, a total of <code>3</code> orbits.</li>
<li><code>L</code> directly orbits <code>K</code> and indirectly orbits <code>J</code>, <code>E</code>, <code>D</code>, <code>C</code>, <code>B</code>, and <code>COM</code>, a total of <code>7</code> orbits.</li>
<li><code>COM</code> orbits nothing.</li>
</ul>
<p>The total number of direct and indirect orbits in this example is <code><em>42</em></code>.</p>
<p><em>What is the total number of direct and indirect orbits</em> in your map data?</p>
</article>


In [62]:
from collections import deque
from functools import reduce

from more_itertools import one


example = """
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
"""


class Orbit:
    def __init__(self, s: str) -> None:
        orbits = defaultdict(set)
        undirected = defaultdict(set)
        for line in s.strip().splitlines():
            center, orbiter = line.split(")")
            orbits[center].add(orbiter)
            undirected[center].add(orbiter)
            undirected[orbiter].add(center)

        self.root = "COM"
        self.orbits = orbits
        self.undirected = undirected

    def total_orbits(self) -> int:
        def count_orbits(count: int, node: str) -> int:
            return count + sum(
                count_orbits(count + 1, child) for child in self.orbits[node]
            )

        return count_orbits(0, self.root)

    def min_orbital_transfers(self, target="SAN") -> int:
        queue = deque(["YOU"])
        seen = set()
        steps = 0
        while queue:
            for _ in range(len(queue)):
                node = queue.popleft()

                if node == target:
                    return steps - 2

                if node in seen:
                    continue

                seen.add(node)

                queue.extend(c for c in self.undirected[node])

            steps += 1

        return -1

    def __repr__(self) -> str:
        return f"root={self.root} and orbits={self.orbits}"


print(f"Example: {Orbit(example).total_orbits()} should be 42")

Example: 42 should be 42


In [63]:
with open("../input/day6.txt") as f:
    puzzle = f.read()

print(f"Part II: {Orbit(puzzle).total_orbits()}")

Part II: 247089


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>247089</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, you just need to figure out how many <em>orbital transfers</em> you (<code>YOU</code>) need to take to get to Santa (<code>SAN</code>).</p>
<p>You start at the object <code>YOU</code> are orbiting; your destination is the object <code>SAN</code> is orbiting. An orbital transfer lets you move from any object to an object orbiting or orbited by that object.</p>
<p>For example, suppose you have the following map:</p>
<pre><code>COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN
</code></pre>
<p>Visually, the above map of orbits looks like this:</p>
<pre><code>                          <em>YOU</em>
                         <em>/</em>
        G - H       <em>J - K</em> - L
       /           <em>/</em>
COM - B - C - <em>D - E</em> - F
               <em>\</em>
                <em>I - SAN</em>
</code></pre>
<p>In this example, <code>YOU</code> are in orbit around <code>K</code>, and <code>SAN</code> is in orbit around <code>I</code>. To move from <code>K</code> to <code>I</code>, a minimum of <code>4</code> orbital transfers are required:</p>
<ul>
<li><code>K</code> to <code>J</code></li>
<li><code>J</code> to <code>E</code></li>
<li><code>E</code> to <code>D</code></li>
<li><code>D</code> to <code>I</code></li>
</ul>
<p>Afterward, the map of orbits looks like this:</p>
<pre><code>        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
                 <em>\</em>
                  <em>YOU</em>
</code></pre>
<p><em>What is the minimum number of orbital transfers required</em> to move from the object <code>YOU</code> are orbiting to the object <code>SAN</code> is orbiting? (Between the objects they are orbiting - <em>not</em> between <code>YOU</code> and <code>SAN</code>.)</p>
</article>

</main>


In [64]:
example_II = """
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN
"""

print(f"Example: {Orbit(example_II).min_orbital_transfers()} should be 4")

Example: 4 should be 4


In [65]:
print(f"Part II: {Orbit(puzzle).min_orbital_transfers()}")

Part II: 442


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>442</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
